In [6]:
import sagemaker
import boto3
import json
from PIL import Image

from botocore.config import Config
from torchvision import transforms
from sagemaker.predictor import Predictor

In [7]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [8]:
val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


def preprocess_image(img_path, transform):
    image = Image.open(img_path).convert('RGB')
    image = transform(image)

    image = image.unsqueeze(0)
    return image


image_path = 'file-003.webp'
input_data = preprocess_image(image_path, val_test_transform)

# Convert the tensor to a list and wrap it in a dictionary
input_list = input_data.tolist()
input_dict = {"inputs": input_list}

# Serialize the dictionary to a JSON string
input_json = json.dumps(input_dict)

In [9]:
config = Config(
    connect_timeout=500,
    read_timeout=200
)

runtime_client = boto3.client("sagemaker-runtime", config=config)

endpoint_name = "pytorch-inference-2024-06-21-22-50-29-291"

predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session
)

In [10]:
# Perform prediction
prediction = predictor.predict(input_json, initial_args={'ContentType': 'application/json'})
print(prediction)
print("Success!!!!!")

b'{"outputs": 1}'
Success!!!!!
